In [1]:
import skrf as rf
import numpy as np
import os

matplotlib not found while setting up plotting


In [13]:
#The list of files from the "good" dir

file_list_good = [f for f in os.listdir(os.path.join('good', 'bga125n6')) if os.path.isfile(os.path.join('good', 'bga125n6', f)) and f.casefold().endswith('.s2p')]
file_list_wrong = [f for f in os.listdir(os.path.join('wrong', 'bga125n6')) if os.path.isfile(os.path.join('wrong', 'bga125n6', f)) and f.casefold().endswith('.s2p')]

print(file_list_good)

['141201_1V2.s2p', '141201_1V8.s2p', '1V8_BGA125N6_23110904.s2p', '1_2V22103.s2p', '1_2V_221001.s2p', '1_8V22103.s2p', '1_8V_221001.s2p', '2V8_BGA125N6_23110904.s2p', '2_8V22103.s2p', '2_8V_221001.s2p', '300602_1V2_BGA125N6.S2P', '300602_1V8_BGA125N6.S2P', '300602_2V8_BGA125N6.S2P', '300603_1V8_BGA125N6.S2P', 'Spara_1v8.s2p']


In [57]:
s_params_g = []
freq_points_g = []
labels = []

s11_points_g = []
s21_points_g = []
s12_points_g = []
s22_points_g = [] 

for file in file_list_good:
    # Read in the S2P file using scikit-rf
    network = rf.Network(os.path.join('good', 'bga125n6', file))

    # Frequency points
    freq = network.f
    for freqpoint in freq:
        labels.append(1) # assume all devices in the "good" directory are within range (this is checked)
        freq_points_g.append(freqpoint)
    # S11, S21, S12, and S22 values
    s11 = network.s[:, 0, 0]
    for s11point in s11:
        s11_points_g.append(s11point)
    s21 = network.s[:, 1, 0]
    for s21point in s21:
        s21_points_g.append(s21point)
    s12 = network.s[:, 0, 1]
    for s12point in s12:
        s12_points_g.append(s12point)
    s22 = network.s[:, 1, 1]
    for s22point in s22:
        s22_points_g.append(s22point)
    
    # Append the extracted values to the list for all sparameters
    s_params_g.append((s11, s21, s12, s22)) 
    

s_params_w = []
freq_points_w = []
labels_w = []

s11_points_w = []
s21_points_w = []
s12_points_w = []
s22_points_w = []

for file in file_list_wrong:
    # Read in the S2P file using scikit-rf
    network = rf.Network(os.path.join('wrong', 'bga125n6', file))

    # Frequency points
    freq = network.f
    for freqpoint in freq:
        labels.append(0) # assume all devices in the "wrong" directory are within range (this is checked)
        freq_points_w.append(freqpoint)
    # S11, S21, S12, and S22 values
    s11 = network.s[:, 0, 0]
    for s11point in s11:
        s11_points_w.append(s11point)
    s21 = network.s[:, 1, 0]
    for s21point in s21:
        s21_points_w.append(s21point)
    s12 = network.s[:, 0, 1]
    for s12point in s12:
        s12_points_w.append(s12point)
    s22 = network.s[:, 1, 1]
    for s22point in s22:
        s22_points_w.append(s22point)
    
    # Append the extracted values to the list for all sparameters
    s_params_w.append((s11, s21, s12, s22))



input_data = []
for i, freq in enumerate(freq_points_g):
    input_data.append([freq, s11_points_g[i], s12_points_g[i], s21_points_g[i], s22_points_g[i]])
for i, freq in enumerate(freq_points_w):
    input_data.append([freq, s11_points_w[i], s12_points_w[i], s21_points_w[i], s22_points_w[i]])
    

In [58]:
# Normalize Data (with complex value Sparameters)

# Normalize the frequency values
freq_values = [entry[0] for entry in input_data]
freq_values_normalized = [freq / max(freq_values) for freq in freq_values]

# Normalize the S-parameters
s_params = [entry[1:] for entry in input_data]
s_params_normalized = []
for s_param in s_params:
    s_param_mag = [np.abs(val) for val in s_param]
    s_param_phase = [np.angle(val) for val in s_param]
    s_param_mag_normalized = [mag / max(s_param_mag) for mag in s_param_mag]
    s_param_phase_normalized = [(phase + np.pi) / (2 * np.pi) for phase in s_param_phase]
    s_params_normalized.append(s_param_mag_normalized + s_param_phase_normalized)

# Combine the normalized frequency values and S-parameters
normalized_data = [[freq] + s_param for freq, s_param in zip(freq_values_normalized, s_params_normalized)]

print(normalized_data[0:10])
print(len(normalized_data))
print(len(labels))

[[1.5e-05, 0.9608299614605524, 0.009698247982418574, 0.0025964632492172864, 1.0, 0.5025273498170555, 0.5226550467594444, 0.5530216865988888, 0.49888443057420834], [0.000514985, 0.9844714398343205, 0.000475368859110444, 0.007335223659695249, 1.0, 0.4972650227080722, 0.8608795372694446, 0.22493637357222218, 0.4946553646119444], [0.00101497, 0.9853333262563305, 0.0005501203552580958, 0.014616167788545293, 1.0, 0.4942933006726389, 0.4623181931622222, 0.2505043497961111, 0.48963662165375], [0.001514955, 0.9856739082165006, 0.0009507854182761861, 0.021955989565276352, 1.0, 0.4916838294187778, 0.892276980025, 0.24981631786416664, 0.48408690554255557], [0.00201494, 0.988439745685883, 0.0004152583718258047, 0.029564373386121647, 1.0, 0.4889365683806111, 0.8120065116, 0.2539619152038889, 0.47929473226244446], [0.002514925, 0.985078960939719, 0.0010676852599989012, 0.0370561033885269, 1.0, 0.48602323780913886, 0.8358304680055555, 0.25878735828666666, 0.47411626186180555], [0.00301491, 0.986045058

In [63]:
n_samples = 18500

true_samples = []
neg_samples = []
for noramlized_sample, label in zip(normalized_data, labels):
    if label == 1:
        true_samples.append(noramlized_sample)
    else:
        neg_samples.append(noramlized_sample)

true_samples = true_samples[:n_samples]
neg_samples = neg_samples[:n_samples]

true_labels = np.ones((n_samples))
neg_labels = np.zeros((n_samples))


In [70]:
from sklearn.model_selection import train_test_split

X = true_samples + neg_samples
y = np.concatenate([true_labels,neg_labels])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, shuffle=True)

In [71]:
import pandas as pd
pd.DataFrame({'labels':y_train}).value_counts()

labels
1.0       15771
0.0       15679
Name: count, dtype: int64

In [72]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [82]:

lamda = 1e-4

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(9,), kernel_regularizer=tf.keras.regularizers.l2(lamda)),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(lamda)),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(lamda))
])

print(model.input_shape)


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

(None, 9)


c:\Users\CasadoMoran\Desktop\Modeling_deembedding\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [83]:
# Train the model
model.fit(np.array(X_train), y_train, epochs=25, batch_size=256, validation_split=0.2)

Epoch 1/25
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6108 - loss: 0.6484 - val_accuracy: 0.5986 - val_loss: 0.6054
Epoch 2/25
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6101 - loss: 0.6061 - val_accuracy: 0.6153 - val_loss: 0.5952
Epoch 3/25
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6309 - loss: 0.5967 - val_accuracy: 0.5975 - val_loss: 0.5891
Epoch 4/25
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6240 - loss: 0.5939 - val_accuracy: 0.7010 - val_loss: 0.5863
Epoch 5/25
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6584 - loss: 0.5854 - val_accuracy: 0.6846 - val_loss: 0.5725
Epoch 6/25
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6794 - loss: 0.5737 - val_accuracy: 0.7078 - val_loss: 0.5633
Epoch 7/25
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6887 - loss: 0.5650 - val_accuracy: 0.7068 - val_loss: 0.5534
Epoch 8/25
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7090 - loss: 0.5539 - val_accuracy: 0.7169 - val_loss:

In [84]:
Y_pred = model.predict(np.array(X_test))

174/174 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [88]:
Y_pred = Y_pred > 0.5 

In [96]:
np.sum(Y_pred[0] == y_test.astype(bool))/(len(Y_pred))

0.4917117117117117